In [16]:
import pymysql

In [17]:
con= pymysql.connect(
    host="localhost",
    user="root",
    password="12345",
    autocommit=True
    )

In [18]:
mycursor=con.cursor()

In [19]:
mycursor.execute("CREATE DATABASE AP_BUS_DETAILS")

1

In [20]:
mycursor.execute("SHOW DATABASES")
for x in mycursor:
    print(x)

('ap_bus_details',)
('bus_details',)
('information_schema',)
('mysql',)
('performance_schema',)
('sakila',)
('sys',)
('world',)


In [21]:
mycursor.execute("USE AP_BUS_DETAILS")

0

In [22]:
mycursor.execute("""
CREATE TABLE AP_bus (
    id INT AUTO_INCREMENT PRIMARY KEY,
    route_name VARCHAR(255),
    route_link VARCHAR(500),
    bus_name VARCHAR(255),
    bus_type VARCHAR(255),
    duration VARCHAR(100),
    price VARCHAR(50),
    rating FLOAT,
    no_of_ratings FLOAT,
    review_status VARCHAR(50)
)
""")

0

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

# URL of APSRTC routes page
URL = "https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile"

# Initialize driver
def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

# Load page
def load_page(driver, url):
    driver.get(url)
    time.sleep(5)

# Scrape bus routes (links + names)
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Scroll to end of bus list
def scroll_until_end(driver):
    scrolling = True
    while scrolling:
        old_page_source = driver.page_source
        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(3)
        new_page_source = driver.page_source
        if new_page_source == old_page_source:
            scrolling = False

# Scrape bus details for one route
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)

       
        try:
             wait = WebDriverWait(driver, 10)             
             apsrtc = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[text()='APSRTC Buses']")))
             apsrtc.click()
             time.sleep(5)
        except:
            print(f"No View Buses button for {route_name}")

        scroll_until_end(driver)

        # Find bus cards
        bus_cards = driver.find_elements(By.XPATH, "//li[contains(@class,'tupleWrapper')]")
        print(f"{len(bus_cards)} buses found for {route_name}")

        bus_details = []
        for bus in bus_cards:
            try:
                name = bus.find_element(By.XPATH, ".//div[contains(@class,'travelsName')]").text
                bus_type = bus.find_element(By.XPATH, ".//p[contains(@class,'busType')]").text
                rating = bus.find_element(By.XPATH, ".//div[contains(@class,'rating')]").text
                price = bus.find_element(By.XPATH, ".//p[contains(@class,'finalFare')]").text
                duration = bus.find_element(By.XPATH, ".//p[contains(@class,'duration')]").text

                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": name,
                    "Bus_Type": bus_type,
                    "Duration": duration,
                    "Rating": rating,
                    "Price": price
                }
                bus_details.append(bus_detail)
            except Exception as e:
                print(f"Error extracting bus in {route_name}: {e}")

        return bus_details

    except Exception as e:
        print(f"Error accessing {url}: {e}")
        return []

# Main execution
driver = initialize_driver()
load_page(driver, URL)

all_bus_details = []

# Scrape all routes and their details
try:
    all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
    for link, name in zip(all_bus_routes_link, all_bus_routes_name):
        print(f"Scraping route: {name}")
        route_bus_details = scrape_bus_details(driver, link, name)
        if route_bus_details:
            all_bus_details.extend(route_bus_details)

except Exception as e:
    print(f"Error in scraping routes: {e}")

# Convert to DataFrame and save
df = pd.DataFrame(all_bus_details)
df.to_csv('apsrtc_all_routes_scraped.csv', index=False)
print("Data saved successfully to apsrtc_all_routes_scraped.csv")

driver.quit()


Scraping route: Hyderabad to Vijayawada
136 buses found for Hyderabad to Vijayawada
Scraping route: Vijayawada to Hyderabad
125 buses found for Vijayawada to Hyderabad
Scraping route: Kakinada to Visakhapatnam
15 buses found for Kakinada to Visakhapatnam
Scraping route: Chittoor (Andhra Pradesh) to Bangalore
32 buses found for Chittoor (Andhra Pradesh) to Bangalore
Scraping route: Bangalore to Tirupati
54 buses found for Bangalore to Tirupati
Scraping route: Hyderabad to Ongole
32 buses found for Hyderabad to Ongole
Scraping route: Bangalore to Kadapa
35 buses found for Bangalore to Kadapa
Scraping route: Ongole to Hyderabad
31 buses found for Ongole to Hyderabad
Scraping route: Kadapa to Bangalore
37 buses found for Kadapa to Bangalore
Scraping route: Madanapalli to Bangalore
12 buses found for Madanapalli to Bangalore
Data saved successfully to apsrtc_all_routes_scraped.csv


In [3]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Duration,Rating,Price
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 35188,VENNELA (A.C. SLEEPER),6h 5m,4\n33,₹781
1,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3563,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",6h 15m,3.6\n29,₹436
2,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 35266,VENNELA (A.C. SLEEPER),6h 5m,3.7\n31,₹781
3,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3590,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",6h 15m,3.5\n22,₹436
4,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3058,VENNELA (A.C. SLEEPER),6h 5m,3.9\n43,₹781
...,...,...,...,...,...,...,...
504,Madanapalli to Bangalore,https://www.redbus.in/bus-tickets/madanapalli-...,APSRTC - 6006,INDRA(A.C. Seater),2h 30m,3.5\n97,₹279
505,Madanapalli to Bangalore,https://www.redbus.in/bus-tickets/madanapalli-...,APSRTC - 5675,Express Non AC Seater 2+3,4h,3.5\n50,₹173
506,Madanapalli to Bangalore,https://www.redbus.in/bus-tickets/madanapalli-...,APSRTC - 7594,Express Non AC Seater 2+3,4h,4\n21,₹173
507,Madanapalli to Bangalore,https://www.redbus.in/bus-tickets/madanapalli-...,APSRTC - 5759,Express Non AC Seater 2+3,4h,3.6\n78,₹173


In [ ]:
import re
import pandas as pd



# Function to extract rating and no_of_ratings with safe checks
def extract_rating_and_count(text):
    matches = re.findall(r'\d+(?:\.\d+)?', str(text))

    # Extract first as rating (float) if exists and not empty
    rating = float(matches[0]) if len(matches) >= 1 and matches[0] != '' else None
    
    # Extract second as no_of_ratings (int) if exists and not empty
    no_of_ratings = int(matches[1]) if len(matches) >= 2 and matches[1] != '' else None
    
    return pd.Series([rating, no_of_ratings])

# Apply function to DataFrame
df[['rating_value', 'No_of_Ratings']] = df['Rating'].apply(extract_rating_and_count)

# Define review_status based on rating_value
def status(rating):
    if rating is None:
        return 'Unknown'
    elif rating >= 4.5:
        return 'Excellent Service'
    elif rating >= 4:
        return 'Good Service'
    elif rating >=3:
        return 'Average'
    else:
        return 'Below Average'

df['Review_Status'] = df['rating_value'].apply(status)


df.drop(columns=['Rating'], inplace=True)


df.rename(columns={'rating_value': 'Rating'}, inplace=True)




                   Route_Name  \
0     Hyderabad to Vijayawada   
1     Hyderabad to Vijayawada   
2     Hyderabad to Vijayawada   
3     Hyderabad to Vijayawada   
4     Hyderabad to Vijayawada   
..                        ...   
504  Madanapalli to Bangalore   
505  Madanapalli to Bangalore   
506  Madanapalli to Bangalore   
507  Madanapalli to Bangalore   
508  Madanapalli to Bangalore   

                                            Route_Link        Bus_Name  \
0    https://www.redbus.in/bus-tickets/hyderabad-to...  APSRTC - 35188   
1    https://www.redbus.in/bus-tickets/hyderabad-to...   APSRTC - 3563   
2    https://www.redbus.in/bus-tickets/hyderabad-to...  APSRTC - 35266   
3    https://www.redbus.in/bus-tickets/hyderabad-to...   APSRTC - 3590   
4    https://www.redbus.in/bus-tickets/hyderabad-to...   APSRTC - 3058   
..                                                 ...             ...   
504  https://www.redbus.in/bus-tickets/madanapalli-...   APSRTC - 6006   
505  https:

In [8]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Duration,Price,Rating,No_of_Ratings,Review_Status
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 35188,VENNELA (A.C. SLEEPER),6h 5m,₹781,4.0,33.0,Good Service
1,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3563,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",6h 15m,₹436,3.6,29.0,Average
2,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 35266,VENNELA (A.C. SLEEPER),6h 5m,₹781,3.7,31.0,Average
3,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3590,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",6h 15m,₹436,3.5,22.0,Average
4,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3058,VENNELA (A.C. SLEEPER),6h 5m,₹781,3.9,43.0,Average
...,...,...,...,...,...,...,...,...,...
504,Madanapalli to Bangalore,https://www.redbus.in/bus-tickets/madanapalli-...,APSRTC - 6006,INDRA(A.C. Seater),2h 30m,₹279,3.5,97.0,Average
505,Madanapalli to Bangalore,https://www.redbus.in/bus-tickets/madanapalli-...,APSRTC - 5675,Express Non AC Seater 2+3,4h,₹173,3.5,50.0,Average
506,Madanapalli to Bangalore,https://www.redbus.in/bus-tickets/madanapalli-...,APSRTC - 7594,Express Non AC Seater 2+3,4h,₹173,4.0,21.0,Good Service
507,Madanapalli to Bangalore,https://www.redbus.in/bus-tickets/madanapalli-...,APSRTC - 5759,Express Non AC Seater 2+3,4h,₹173,3.6,78.0,Average


In [ ]:
df.isnull().sum()


Route_Name        0
Route_Link        0
Bus_Name          0
Bus_Type          0
Duration          0
Price             0
Rating            0
No_of_Ratings    37
Review_Status     0
dtype: int64

In [10]:
df['No_of_Ratings'].fillna(0, inplace=True)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_4952\2493728059.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['No_of_Ratings'].fillna(0, inplace=True)


In [11]:
df.isnull().sum()

Route_Name       0
Route_Link       0
Bus_Name         0
Bus_Type         0
Duration         0
Price            0
Rating           0
No_of_Ratings    0
Review_Status    0
dtype: int64

In [14]:
df.duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
504    False
505    False
506    False
507    False
508    False
Length: 509, dtype: bool

In [15]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Duration,Price,Rating,No_of_Ratings,Review_Status
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 35188,VENNELA (A.C. SLEEPER),6h 5m,₹781,4.0,33.0,Good Service
1,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3563,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",6h 15m,₹436,3.6,29.0,Average
2,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 35266,VENNELA (A.C. SLEEPER),6h 5m,₹781,3.7,31.0,Average
3,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3590,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",6h 15m,₹436,3.5,22.0,Average
4,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3058,VENNELA (A.C. SLEEPER),6h 5m,₹781,3.9,43.0,Average
...,...,...,...,...,...,...,...,...,...
504,Madanapalli to Bangalore,https://www.redbus.in/bus-tickets/madanapalli-...,APSRTC - 6006,INDRA(A.C. Seater),2h 30m,₹279,3.5,97.0,Average
505,Madanapalli to Bangalore,https://www.redbus.in/bus-tickets/madanapalli-...,APSRTC - 5675,Express Non AC Seater 2+3,4h,₹173,3.5,50.0,Average
506,Madanapalli to Bangalore,https://www.redbus.in/bus-tickets/madanapalli-...,APSRTC - 7594,Express Non AC Seater 2+3,4h,₹173,4.0,21.0,Good Service
507,Madanapalli to Bangalore,https://www.redbus.in/bus-tickets/madanapalli-...,APSRTC - 5759,Express Non AC Seater 2+3,4h,₹173,3.6,78.0,Average


In [24]:
insert_query = """
INSERT INTO AP_bus (
    route_name, route_link, bus_name, bus_type, duration,
    price, rating, no_of_ratings, review_status
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Iterate and insert each row
for index, row in df.iterrows():
    mycursor.execute(insert_query, (
        row['Route_Name'],
        row['Route_Link'],
        row['Bus_Name'],
        row['Bus_Type'],
        row['Duration'],
        row['Price'],
        row['Rating'],           # renamed in your cleaning script
        row['No_of_Ratings'],
        row['Review_Status']
    ))

# Commit transaction
con.commit()